In [ ]:
!pip install torch torchaudio demucs numpy tqdm


In [ ]:
!pip uninstall -y demucs
!pip install git+https://github.com/facebookresearch/demucs


Found existing installation: demucs 4.1.0a2
Uninstalling demucs-4.1.0a2:
  Successfully uninstalled demucs-4.1.0a2
  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-req-build-v44tpv63
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-req-build-v44tpv63
  Resolved https://github.com/facebookresearch/demucs to commit e976d93ecc3865e5757426930257e200846a520a
  Preparing metadata (setup.py) ... done
  Created wheel for demucs: filename=demucs-4.1.0a2-py3-none-any.whl size=83540 sha256=8f7bba5e01847ca7e10a26674a04c0c14eb7db81c7e9bf6cdd0e33e4d9340511
  Stored in directory: /tmp/pip-ephem-wheel-cache-dprwon85/wheels/3c/82/96/63388ba864e9b7e19fd05c805a06278b4357e4b54d67e4e58c
Successfully built demucs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from demucs.htdemucs import HTDemucs  # Correct import for Demucs model


In [ ]:
train_dataset_path = "/content/drive/MyDrive/New/Demucs_Training_Dataset/train"  # Change this to your training dataset folder
val_dataset_path = "/content/drive/MyDrive/New/Demucs_Training_Dataset/valid"  # Change this to your validation dataset folder


In [ ]:
!pip uninstall -y demucs
!pip install git+https://github.com/facebookresearch/demucs
# Try importing the correct model
from demucs.htdemucs import HTDemucs  # Latest architecture
# or
from demucs.demucs import Demucs  # Older model
!python3 -c "import demucs; print(demucs.__file__)"


Found existing installation: demucs 4.1.0a2
Uninstalling demucs-4.1.0a2:
  Successfully uninstalled demucs-4.1.0a2
  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-req-build-wodiyz70
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-req-build-wodiyz70
  Resolved https://github.com/facebookresearch/demucs to commit e976d93ecc3865e5757426930257e200846a520a
  Preparing metadata (setup.py) ... done
  Created wheel for demucs: filename=demucs-4.1.0a2-py3-none-any.whl size=83540 sha256=fbbacf1fe6abef82b71c4c6f802269952707950b2b1df4e4e2784f31f3e1b2c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-7xaxn_q1/wheels/3c/82/96/63388ba864e9b7e19fd05c805a06278b4357e4b54d67e4e58c
Successfully built demucs
/usr/local/lib/python3.11/dist-packages/demucs/__init__.py


In [ ]:
import torch
import torchaudio
import os
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from demucs import Demucs

# Custom dataset for loading audio files
class AudioDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.files = [f for f in os.listdir(root_dir) if f.endswith('.wav')]  # Load .wav files

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.root_dir, self.files[idx])
        waveform, _ = torchaudio.load(file_path)  # Load audio
        return waveform  # Return audio tensor

# Training function
def train_demucs(model, dataloader, optimizer, epochs, device):
    model.train()
    loss_fn = torch.nn.MSELoss()  # Mean Squared Error for separation loss

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs = inputs.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)  # Pass through model
            loss = loss_fn(outputs, inputs)  # Compute loss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss / len(dataloader)}")

# Main function to load data and train the model
def retrain_demucs(train_dir, val_dir, epochs, batch_size, learning_rate, model_name):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Training on {device}")

    # Load training and validation datasets
    train_dataset = AudioDataset(train_dir)
    val_dataset = AudioDataset(val_dir)

    # Combine both datasets for training
    full_dataset = ConcatDataset([train_dataset, val_dataset])
    dataloader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

    # Load Demucs model
    model = Demucs()  # Initialize Demucs model
    model.to(device)

    # Define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model on both train & validation datasets
    train_demucs(model, dataloader, optimizer, epochs, device)

    # Save trained model to Google Drive
    save_path = f"/content/drive/MyDrive/{model_name}.pth"
    torch.save(model.state_dict(), save_path)
    print(f"Training complete. Model saved as {save_path}")

# Set dataset paths (update these if needed)
train_dataset_path = "/content/drive/MyDrive/Dataset/train"  # Change this to your training dataset folder
val_dataset_path = "/content/drive/MyDrive/Dataset/val"  # Change this to your validation dataset folder

# Run training
retrain_demucs(
    train_dir=train_dataset_path,
    val_dir=val_dataset_path,
    epochs=10,  # Change to desired number of epochs
    batch_size=16,
    learning_rate=3e-4,
    model_name="demucs_colab"
)


ImportError: cannot import name 'Demucs' from 'demucs' (/usr/local/lib/python3.11/dist-packages/demucs/__init__.py)

In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/demucs_colab.pth")
